In [ ]:
import os
import json
import numpy as np
import pandas as pd
from glob import glob

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from matplotlib.ticker import FuncFormatter

matplotlib.rcParams.update({'font.size': 22})

In [ ]:
PLOT_DIR = './plots/'
os.makedirs(PLOT_DIR, exist_ok=True)

In [ ]:
PATH = './runs/noisy/*.json'

In [ ]:
df = pd.DataFrame()

for fn in glob(PATH):
    g = os.path.basename(fn).split('.')[0]
    ds, tf, m, seed = g.split('__')

    with open(fn, 'r') as f:
        jdir = json.load(f)
        data_keys = list(jdir.keys())
        data_keys_diff = [l + '_diff' for l in list(jdir.keys())]
    jdir['method'] = m
    jdir['dataset'] = ds
    jdir['transformation'] = tf
    jdir['seed'] = int(seed)
    tmp_df = pd.DataFrame([jdir])
    tmp_df[data_keys_diff] = tmp_df[data_keys]

    df = pd.concat([df, tmp_df])

for k in data_keys_diff:
    df[k] = df[k] - df['0']

In [ ]:
def percentage(x, pos):
    return f'{x:.0f}%'

for ds in sorted(df['dataset'].unique()):
    for tf in sorted(df['transformation'].unique()):
        print(ds, tf)
        fig, axs = plt.subplots(nrows=2, figsize=(10, 7), sharex=True)  # Create subplots with shared x-axis

        ax1, ax2 = axs  # Separate axes for the two plots
        x = [float(d) for d in data_keys]

        handles = []
        for m in sorted(df['method'].unique()):
            tmp_df = df.query(f'`dataset`=="{ds}" & `transformation`=="{tf}" & `method`=="{m}"')

            y = tmp_df[data_keys].mean()
            std = tmp_df[data_keys].std()
            
            r_upper = y + std
            r_lower = y - std

            l, = ax1.plot(x, y, label=m, linestyle='-', marker='x')
            ax1.fill_between(x, r_upper, r_lower, alpha=0.3)
            handles.append(l)

            y_diff = tmp_df[data_keys_diff].mean()
            std_diff = tmp_df[data_keys_diff].std()
            
            r_upper_diff = y_diff + std_diff
            r_lower_diff = y_diff - std_diff
            
            ax2.plot(x, y_diff, label=m, linestyle='-', marker='x')
            ax2.fill_between(x, r_upper_diff, r_lower_diff, alpha=0.3)
        
        if tf != 'Scale':
            handles = handles[3:]
        else:
            handles = handles[:3]
            ax1.set_ylabel('Accuracy')
            ax2.set_ylabel('Accuracy Difference')
        ax1.set_xlabel('Percentage of added Noise')
        ax1.xaxis.set_label_position('top') 
        ax2.legend(handles=handles, loc='upper center', ncol=3, bbox_to_anchor=(0.5, -.09), fancybox=False, frameon=False)
        ax2.xaxis.set_major_formatter(FuncFormatter(percentage))

        plt.savefig(f'{PLOT_DIR}/{ds}_{tf}_noisy_plots.pdf', bbox_inches='tight')
        plt.show()